In [1]:
#%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import time
import random
import copy
from copy import deepcopy
import threading

from train_utils import *

import matplotlib.pyplot as plt
import cv2

In [3]:
import gym3
from procgen import ProcgenGym3Env

In [4]:
bs = 1

# num_levels=1, start_level=6
env = ProcgenGym3Env(num=bs, env_name="testgame", render_mode='rgb_array')
env = gym3.ViewerWrapper(env, info_key="rgb")

building procgen...done


In [5]:
device = 'cuda'
m = VizCNN(use_rnn=False).to(device);
m.load_state_dict(torch.load("m_cons_3.27.torch"))
m.eval()

VizCNN(
  (pooler): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act): ReLU()
  (conv_1a): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_2a): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (bn2a): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_2b): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn2b): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_3a): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_4a): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=5189, out_features=512, bias=True)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (

In [6]:
hidden = get_hidden(1)

In [22]:
%%time

use_autopilot = True

s = np.array([[.1, .2] for _ in range(bs)], dtype=np.float32)
daggerized_controls = s


seq_len = 1000
DAGGER_CADENCE = 100
DAGGER_DURATION = 10
do_dagger = False
dagger_counter = 0
TRAINING_WHEELS_WINDOW = 20
use_training_wheels = False

act_grads = []
imgs = []
salmaps = []

#with torch.no_grad():
for i in range(seq_len):
    env.act(s)
    rew, obs, first = env.observe()
    img = obs['rgb']
    info = env.get_info()

    aux = np.array([[e[a] for a in aux_properties] for e in info])
    autopilot_controls = np.array([[e['autopilot_steer'], e['autopilot_throttle']] for e in info])

    front = torch.from_numpy(img.astype(np.float32)/255.).unsqueeze(0).permute(0,1,4,2,3)
    aux = torch.from_numpy(aux.astype(np.float32)).unsqueeze(0)

    if use_autopilot:
        if i % DAGGER_CADENCE == 0:
            print('SETTING DAGGER')
            dagger_counter = 0
            steer_aug = random.uniform(-.3, .3)
            #throttle_aug = random.uniform(.5, 1.5)
            daggerized_controls = np.array([[c[0]+steer_aug, c[1]] for c in autopilot_controls])
            do_dagger = True

        if do_dagger:
            s = daggerized_controls
            dagger_counter+=1
        else:
            s = autopilot_controls

        if dagger_counter == DAGGER_DURATION:
            do_dagger = False

    else:
        front = front.to(device)
        aux = aux.to(device)
        out,hidden, salmap = m(front, aux, hidden, return_salmap=True, register_activations=True)
        out[:,:,1]=.6
        s = out.squeeze(0).cpu().detach().numpy()
        
        
        """        baseline = torch.zeros_like(front)
        steps = 100
        grads = []

        for i in range(0,steps+1):
            mixed_img = baseline + (float(i)/steps)*(front-baseline)
            m.zero_grad()
            out, hidden, salmap = m(mixed_img, aux, hidden, return_salmap=True, register_activations=True)
            out[0][0][0].backward(retain_graph=m.use_rnn) # Steer
            g = m.get_activations_gradient()
            g = g[0].mean(0).cpu().numpy()
            grads.append(g)

        avg_grads = np.stack(grads).mean(0)
        act_grad = avg_grads
        
        salmap = salmap[0].mean(0)
        
        act_grads.append(act_grad * salmap)
        #act_grads.append(salmap) # This by itself is pretty nice
        
        imgs.append(img[0])"""

    if use_training_wheels and i < TRAINING_WHEELS_WINDOW:
        s = autopilot_controls
        

SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
CPU times: user 481 ms, sys: 53 ms, total: 534 ms
Wall time: 389 ms


In [7]:

def img_cam(act_grad, img, std):
    # Gradients zero to one
    act_grad = cv2.resize(act_grad, (img.shape[0],img.shape[1]))
    mask = np.where(act_grad, (abs(act_grad)>std*2), 0)
    mask = np.expand_dims(mask, -1)
    
    act_grad -= act_grad.min()
    act_grad = act_grad / act_grad.max()
    
    # Make a three-channel heatmap out of the one channel gradients
    heatmap = cv2.applyColorMap(np.uint8(255 * act_grad), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    heatmap = heatmap * mask
    heatmap = heatmap * 255
    
    #img = img / 255
    
    cam = heatmap*.5 + img
    
    #cam = cam / cam.max()
    #cam = cam * 255
    cam = np.clip(cam, 0, 255)
    cam = cam.astype(np.uint8)
    
    return cam

In [8]:
std = np.array(act_grads[20:]).std()
img_cams = []
for img, act_grad in zip(imgs,act_grads):
    img_cams.append(img_cam(act_grad,img,std))

/home/beans/anaconda3/envs/procgen/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/beans/anaconda3/envs/procgen/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/beans/anaconda3/envs/procgen/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
img1 = img_cams[0]
height , width , layers = img1.shape
fps = 10

video = cv2.VideoWriter('cams.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (width,height))

for i in range(len(img_cams)-1):
    img = img_cams[i] # cv2 expects out of 255, integers
    video.write(img)
    
cv2.destroyAllWindows()
video.release()

In [10]:
salmap.shape

(1, 16, 29, 29)